In [ ]:
!pip install transformers==4.21.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 10.4 MB/s 
     |████████████████████████████████| 6.6 MB 65.3 MB/s 
     |████████████████████████████████| 182 kB 79.7 MB/s 


In [ ]:
import transformers

In [ ]:
print(transformers.__version__)

4.21.2


In [ ]:
import pandas as pd
import numpy as np
import datetime
import random
import time
import re

import torch

from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report


from collections import Counter
from transformers import BertForSequenceClassification, Trainer, TrainingArguments
from transformers import get_linear_schedule_with_warmup
from transformers import AutoConfig, AutoTokenizer, AutoModel, TFAutoModel
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import logging
from tensorflow.keras.utils import to_categorical
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from torch.utils.data import TensorDataset, random_split

from termcolor import colored
from tqdm import tqdm

In [ ]:
corpus=pd.read_csv('/content/drive/MyDrive/snappfood_comments_preprocessed.csv')

In [ ]:
texts=corpus['Cleaned'].values

In [ ]:
labels = corpus['label_id'].values

In [ ]:
validation_ratio = 10/100

In [ ]:
test_texts = texts[int(len(texts)-(len(texts)*validation_ratio)):]

In [ ]:
test_labels = labels[int(len(texts)-(len(texts)*validation_ratio)):]

In [ ]:
train_texts = texts[:int(len(texts)-(len(texts)*validation_ratio))]

In [ ]:
train_labels = labels[:int(len(texts)-(len(texts)*validation_ratio))]

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(train_texts, train_labels, test_size=0.2, random_state=42)

In [ ]:
categorical_y_train = to_categorical(y_train, num_classes=2, dtype="float32")
categorical_y_val = to_categorical(y_val, num_classes=2, dtype="float32")
categorical_test_labels = to_categorical(test_labels, num_classes=2, dtype="float32")

In [ ]:
model_name = 'HooshvareLab/bert-fa-zwnj-base'

In [ ]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(model_name)

In [ ]:
train_encodings = tokenizer(list(X_train), truncation=True, padding=True)
val_encodings = tokenizer(list(X_val), truncation=True, padding=True)
test_encodings = tokenizer(list(test_texts), truncation=True, padding=True)

In [ ]:
class SFDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = SFDataset(train_encodings, categorical_y_train)
val_dataset = SFDataset(val_encodings, categorical_y_val)
test_dataset = SFDataset(test_encodings, categorical_test_labels)

In [ ]:
training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/results',          
    num_train_epochs=3,             
    per_device_train_batch_size=16,  
    per_device_eval_batch_size=64,   
    warmup_steps=500,                
    weight_decay=0.01,               
    logging_dir='/content/drive/MyDrive/logs',            
    logging_steps=10,
    
)

model = BertForSequenceClassification.from_pretrained(model_name)

trainer = Trainer(
    model=model,                         
    args=training_args,                  
    train_dataset=train_dataset,         
    eval_dataset=val_dataset,            
)

Some weights of the model checkpoint at HooshvareLab/bert-fa-zwnj-base were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at HooshvareLab/b

In [ ]:
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 50025
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 9381


Step,Training Loss
10,0.713100
20,0.713100
30,0.696800
40,0.689400
50,0.672500
60,0.654000
70,0.637000
80,0.634500
90,0.547800
100,0.554900


Saving model checkpoint to /content/drive/MyDrive/results/checkpoint-500
Configuration saved in /content/drive/MyDrive/results/checkpoint-500/config.json
Model weights saved in /content/drive/MyDrive/results/checkpoint-500/pytorch_model.bin
Saving model checkpoint to /content/drive/MyDrive/results/checkpoint-1000
Configuration saved in /content/drive/MyDrive/results/checkpoint-1000/config.json
Model weights saved in /content/drive/MyDrive/results/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to /content/drive/MyDrive/results/checkpoint-1500
Configuration saved in /content/drive/MyDrive/results/checkpoint-1500/config.json
Model weights saved in /content/drive/MyDrive/results/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to /content/drive/MyDrive/results/checkpoint-2000
Configuration saved in /content/drive/MyDrive/results/checkpoint-2000/config.json
Model weights saved in /content/drive/MyDrive/results/checkpoint-2000/pytorch_model.bin
Saving model checkpoint to 

TrainOutput(global_step=9381, training_loss=0.32982789338597635, metrics={'train_runtime': 10470.0938, 'train_samples_per_second': 14.334, 'train_steps_per_second': 0.896, 'total_flos': 3.0077524876815e+16, 'train_loss': 0.32982789338597635, 'epoch': 3.0})

In [ ]:
trainer.save_model("/content/drive/MyDrive/fintuned")

Saving model checkpoint to /content/drive/MyDrive/fintuned
Configuration saved in /content/drive/MyDrive/fintuned/config.json
Model weights saved in /content/drive/MyDrive/fintuned/pytorch_model.bin


In [ ]:
model.save_pretrained("/content/drive/MyDrive/fintuned_model")

Configuration saved in /content/drive/MyDrive/fintuned_model/config.json
Model weights saved in /content/drive/MyDrive/fintuned_model/pytorch_model.bin
